# Alpaca Benro Polaris - Data5 Telemetary Plot

This notebook visualizes telemetry from the Benro Polaris mount as well as information from the Alpaca Kalman Filter that has been recorded into the `alpaca.csv` log file. 
It shows the mounts orientation (Az/Alt/Roll), the motor axis angular position Theta (measured and estimated state), the motor axis angular velocity Omega (measured, controlled and estimated state).

To ensure compatible data is captured for this notebook, update the `config.toml` file with the following setting:

```toml
log_performance_data = 5
```

After making this change, restart the Alpaca Driver to begin logging the required telemetry. The `alpaca.csv` should now include lines with the following type of data.
```
2025-07-24T09:48:05.500 INFO ,Dataset,Time,  az,alt,roll,  theta1,theta2,theta3,  state1,state2,state3,  omega1,omega2,omega3,  state4,state5,state6,  oref1,oref2,oref3
2025-07-24T09:48:05.827 INFO ,DATA5,0.0000,  +216.5474,+13.1680,-71.6810,  +130.8607,+72.1790,+76.1554,  +130.8607,+72.1790,+76.1554,  +0.00000,+0.00000,+0.00000, +0.00000,+0.00000,+0.00000,  +1.00000,+0.00000,+0.00000 
2025-07-24T09:48:06.033 INFO ,DATA5,0.2059,  +216.5470,+13.1681,-71.6810,  +130.8603,+72.1791,+76.1554,  +130.8603,+72.1791,+76.1554,  -0.00212,+0.00031,-0.00001, -0.00047,+0.00022,-0.00000,  +1.00000,+0.00000,+0.00000 
2025-07-24T09:48:06.233 INFO ,DATA5,0.4063,  +216.5475,+13.1681,-71.6811,  +130.8607,+72.1792,+76.1554,  +130.8605,+72.1792,+76.1554,  -0.00001,+0.00042,-0.00001, +0.00113,+0.00031,-0.00001,  +1.00000,+0.00000,+0.00000 
```
Timestamp & Metadata
- **Timestamp**: ISO 8601 format (e.g. `2025-07-24T09:48:05.500`) — precise time of the log entry.
- **INFO**: Log level indicator.
- **DATA5**: Tag identifying this line as a telemetry record relevant to the DATA5 format.
- **Elapsed Time**: Seconds since the start of the logging session (e.g. `0.0000`, `0.2059`).

Measured Orientation (Azimuth, Altitude, Roll) based on Quaternion1 updates from Polaris
- **az**: Azimuth angle (degrees) — horizontal rotation relative to true north.
- **alt**: Altitude angle (degrees) — vertical angle above the horizon.
- **roll**: Roll angle (degrees) — rotation around the camera boresight or optical axis.

Measured Motor Angular Position and Velocity (theta and omega) based on Quaternion1 updates from Polaris
- **theta1, theta2, theta3**: Angular positions for each motor axis (degrees). 
- **omega1, omega2, omega3**: Angular velocities (degrees/sec) for each motor axis.

Control Signals based on moveaxis commands, the current SP sent to Alpaca motor speed control or directly to Polaris
- **oref1, oref2, oref3**: Control Signal for Angular Velocities, equiv to omega1,2,3

Internal Kalman Filter State Estimates based on Measurement and Control updates
- **state1, state2, state3**: Estimated angular positions from the Kalman Filter, equivc to theta1, 2, 3.
- **state4, state5, state6**: Estimated angular velocities from the Kalman Filter, equiv to omega1, 2, 3.

In [ ]:
import sys
sys.path.insert(0, '../driver')

In [ ]:
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots
from control import polar_rotation_angle

In [ ]:
def read_data5_logfile(filename):
    # Define the column names matching DATA5 log output
    columns = [
        "timestamp",
        "p_az", "p_alt", "p_roll",
        "theta1_meas", "theta2_meas", "theta3_meas",
        "theta1_state", "theta2_state", "theta3_state",
        "omega1_meas", "omega2_meas", "omega3_meas",
        "omega1_state", "omega2_state", "omega3_state",
        "omega1_ref", "omega2_ref", "omega3_ref"
    ]

    # Read and parse only DATA5 lines
    data = []
    with open(filename) as f:
        for line in f:
            if ",DATA5," in line:
                fields = line.strip().split(",")
                if len(fields) >= 20:
                    values = [float(val) for val in fields[2:]]  # Skip timestamp and DATA5 label
                    data.append(values)

    df = pd.DataFrame(data, columns=columns)

    return df

def plot_axis_speed_measured_reference_state_vs_time(df, axis=0):
    n = axis+1
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        subplot_titles=(f"Theta{n}: Measured vs State", f"Omega{n}: Measured vs State vs Reference"))

    # Top subplot: Theta{n}
    fig.add_trace(go.Scatter(
        y=df[f'theta{n}_meas'],
        mode="markers",
        name=f'Theta{n} Measured',
        marker=dict(color="blue", size=6)
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        y=df[f'theta{n}_state'],
        mode="lines",
        name=f'Theta{n} State',
        line=dict(color="orange")
    ), row=1, col=1)

    # Bottom subplot: Omega{n}
    fig.add_trace(go.Scatter(
        y=df[f'omega{n}_meas'],
        mode="markers",
        name=f'Omega{n} Measured',
        marker=dict(color="green", size=6)
    ), row=2, col=1)

    fig.add_trace(go.Scatter(
        y=df[f'omega{n}_state'],
        mode="lines",
        name=f'Omega{n} State',
        line=dict(color="red")
    ), row=2, col=1)

    fig.add_trace(go.Scatter(
        y=df[f'omega{n}_ref'],
        mode="lines",
        name=f'Omega{n} Reference',
        line=dict(color="purple", dash="dot")
    ), row=2, col=1)

    fig.update_layout(
        height=900,
        title=f'Theta{n} and Omega{n} Diagnostic Overview',
        xaxis2_title="Time (frame)",
        yaxis_title=f'Theta{n} (deg)',
        yaxis2_title=f'Omega{n} (deg/sec)',
        legend_title_text="Signal Source",
    )
    fig.show()



def plot_median_speed_vs_reference(df):
    # Group by omega1_ref and calculate median
    median_df = df.groupby('omega1_ref')['omega1_meas'].median().reset_index()

    # Create figure with scatter + median overlay
    fig = go.Figure()

    # Raw scatter points
    fig.add_trace(go.Scatter(
        x=df['omega1_ref'],
        y=df['omega1_meas'],
        mode='markers',
        name='Measured ω₁',
        marker=dict(color='lightblue', size=6, opacity=0.6)
    ))

    # Median points at each omega_ref
    fig.add_trace(go.Scatter(
        x=median_df['omega1_ref'],
        y=median_df['omega1_meas'],
        mode='markers+lines',
        name='Median ω₁ @ ω₁_ref',
        marker=dict(color='orange', size=8, symbol='diamond'),
        line=dict(color='orange', dash='dash')
    ))

    fig.update_layout(
        title='omega1: Measured vs Reference with Median Overlay',
        xaxis_title='Reference ω₁ (deg/s)',
        yaxis_title='Measured ω₁ (deg/s)',
        template='plotly_dark',
        legend=dict(x=0.01, y=0.99)
    )

    fig.show()



In [ ]:
filename = "../logs/alpaca.csv"

df = read_data5_logfile(filename)
plot_axis_speed_measured_reference_state_vs_time(df, axis=0)

In [ ]:
plot_median_speed_vs_reference(df)